<a href="https://colab.research.google.com/github/CristhianSeverino/medallion-retail-pipeline-dbt-gx/blob/main/dbt_Exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **dbt + Great Expectations Practice – Modern Data Pipeline Demo**

> Created by **Cristhian Calle Severino**  
> Senior Solution Architect | BI Lead | Data Strategist

>>Demonstrating Medallion Architecture, Kimball modeling, automated testing, and proactive data governance with Great Expectations.



**If you find this notebook useful, I'd love to hear your thoughts!**  
**Have fun exploring!** 😊



- **GitHub**: https://github.com/CristhianSeverino  
- **LinkedIn**: https://www.linkedin.com/in/cristhianandrescalleseverino/  
- **Portfolio**: https://sites.google.com/view/cristhiancalle  


## 1. Install Libraries & Dependencies 📦

> Installs required packages (dbt-core, dbt-duckdb, pandas, Great Expectations).

> **Note**: If Colab prompts for a runtime restart after installation, please do so before continuing. This is common when upgrading core libraries. 🚨

In [45]:
# Celda 1: Instalall (dbt, GX, pandas, duckdb)
!pip install --upgrade dbt-core dbt-duckdb great-expectations

#2. **Import Libraries 📚**


In [46]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import duckdb
import great_expectations as gx
import json

## 3. Synthetic Data Generation – Simulated ERP Dataset 🛠️

> In this section, we generate a synthetic ERP dataset to emulate real-world data challenges.  

> The simulated ERP intentionally includes **lack of proper data architecture and governance** (e.g., negative quantities, invalid keys, zero metrics, inconsistencies), allowing us to demonstrate cleaning (silver), aggregation (gold), and quality validation (GX) in the following steps.


In [47]:
# Cell 2: More complex data generation (emulating a realistic ERP with anomalies)
# Fixed typos: import pandas, np.random.poisson, np.random.randint, anomaly_idx, np.random.choice


np.random.seed(42)  # Reproducibilidad

# dim_time:  with holidays flag and week_day

dates = pd.date_range(start='2023-01-01', end='2026-01-15', freq='D')
dim_time = pd.DataFrame({
    'time_id': range(1, len(dates) + 1),
    'date': dates,
    'year': dates.year,
    'month': dates.month,
    'day': dates.day,
    'quarter': dates.quarter,
    'week_day': dates.strftime('%A'),  # Día de la semana
    'is_holiday': np.random.choice([0, 1], len(dates), p=[0.95, 0.05])  # 5% holidays
})

# dim_location
dim_location = pd.DataFrame({
    'location_id': range(1, 11),
    'region': np.random.choice(['North', 'South', 'East', 'West', 'Central'], 10),
    'country': ['USA'] * 10,
    'city': np.random.choice(['New York', 'Miami', 'Chicago', 'LA', 'Dallas'], 10)
})

# dim_customer: Add loyalty_level
dim_customer = pd.DataFrame({
    'customer_id': range(1, 501),  # more customers
    'name': ['Customer ' + str(i) for i in range(1, 501)],
    'segment': np.random.choice(['Consumer', 'Corporate', 'Home Office'], 500),
    'loyalty_level': np.random.choice(['Bronze', 'Silver', 'Gold'], 500, p=[0.6, 0.3, 0.1])
})

# dim_product: add subcategory, cost
dim_product = pd.DataFrame({
    'product_id': range(1, 201),  # more products
    'name': ['Product ' + str(i) for i in range(1, 201)],
    'category': np.random.choice(['Furniture', 'Office Supplies', 'Technology'], 200),
    'subcategory': np.random.choice(['Chairs', 'Desks', 'Phones', 'Binders'], 200),
    'cost': np.random.uniform(5, 300, 200).round(2)  # Calculate profit later
})

# fact_sales: 10k rows, with discounts, tax, timestamps, anomalies (5% negative quantity, 2% invalid region_id, revenue mismatches)
num_sales = 10000
fact_sales = pd.DataFrame({
    'sale_id': range(1, num_sales + 1),
    'time_id': np.random.choice(dim_time['time_id'], num_sales),
    'customer_id': np.random.choice(dim_customer['customer_id'], num_sales),
    'product_id': np.random.choice(dim_product['product_id'], num_sales),
    'location_id': np.random.choice(dim_location['location_id'], num_sales),
    'quantity': np.random.poisson(3, num_sales) + 1,  # Media 3, min 1
    'price': np.random.uniform(10, 500, num_sales).round(2),
    'discount': np.random.uniform(0, 0.2, num_sales).round(2),  # 0-20%
    'tax_rate': np.random.choice([0.05, 0.07, 0.1], num_sales),  # Tax variants
    'sale_timestamp': [datetime.now() - timedelta(days=np.random.randint(0, 1095)) for _ in range(num_sales)]  # Last 3 years
})

# Calculate revenue with discount and tax
fact_sales['revenue_before_discount'] = fact_sales['quantity'] * fact_sales['price']
fact_sales['revenue'] = fact_sales['revenue_before_discount'] * (1 - fact_sales['discount']) * (1 + fact_sales['tax_rate'])

# Add intentional anomalies for testing
anomaly_idx = np.random.choice(num_sales, int(num_sales * 0.05), replace=False)
fact_sales.loc[anomaly_idx, 'quantity'] = fact_sales.loc[anomaly_idx, 'quantity'] * -1  # Negatives
fact_sales.loc[np.random.choice(num_sales, int(num_sales * 0.02)), 'location_id'] = -1  # Invalid ID
fact_sales.loc[np.random.choice(num_sales, int(num_sales * 0.01)), 'revenue'] = 0  # Zero revenue anomalies

# Export
fact_sales.to_csv('fact_sales.csv', index=False)
dim_customer.to_csv('dim_customer.csv', index=False)
dim_product.to_csv('dim_product.csv', index=False)
dim_time.to_csv('dim_time.csv', index=False)
dim_location.to_csv('dim_location.csv', index=False)
print("Datos complejos generados: ", fact_sales.shape, " anomalías incluidas.")

Datos complejos generados:  (10000, 12)  anomalías incluidas.


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


## 4. dbt + DuckDB – Local ELT Engine & Medallion Architecture 🧱

> Here we set up **dbt with DuckDB** as our lightweight, in-memory analytical engine.  

> This combination allows us to implement a full **Medallion Architecture** (Bronze → Silver → Gold) with dimensional modeling (Kimball) in a completely local, reproducible, and zero-cost environment — ideal for prototyping and demos.  

**Why this matters**:  
In production, this exact pipeline scales seamlessly to Snowflake, Redshift, or BigQuery by simply changing the adapter — demonstrating true **portability** and **cloud-agnostic design**, a core principle in modern data platforms.

In [48]:
# Cell 3: Create project and manual configuration (to avoid interactive prompts)


project_dir = '/content/sales_analytics'
os.makedirs(project_dir, exist_ok=True)
os.chdir(project_dir)

# Create dbt_project.yml básico
with open('dbt_project.yml', 'w') as f:
    f.write("""
name: 'sales_analytics'
version: '1.0.0'
config-version: 2

profile: 'sales_analytics'

model-paths: ["models"]
analysis-paths: ["analyses"]
test-paths: ["tests"]
seed-paths: ["seeds"]
macro-paths: ["macros"]
snapshot-paths: ["snapshots"]

target-path: "target"
clean-targets: ["target", "dbt_packages"]

models:
  sales_analytics:
    +materialized: table
""")

# Create profiles.yml
with open('profiles.yml', 'w') as f:
    f.write("""
sales_analytics:
  target: dev
  outputs:
    dev:
      type: duckdb
      path: /content/sales_analytics/sales.duckdb
      threads: 4
""")

print("Proyecto creado manualmente.")
!ls -la

Proyecto creado manualmente.
total 8240
drwxr-xr-x 6 root root    4096 Jan 15 23:24 .
drwxr-xr-x 1 root root    4096 Jan 15 23:21 ..
-rw-r--r-- 1 root root  668521 Jan 15 22:40 dbt_docs.zip
drwxr-xr-x 3 root root    4096 Jan 15 22:40 dbt_packages
-rw-r--r-- 1 root root     358 Jan 16 00:06 dbt_project.yml
-rw-r--r-- 1 root root   16902 Jan 16 00:06 dim_customer.csv
-rw-r--r-- 1 root root     224 Jan 16 00:06 dim_location.csv
-rw-r--r-- 1 root root    8235 Jan 16 00:06 dim_product.csv
-rw-r--r-- 1 root root   41269 Jan 16 00:06 dim_time.csv
-rw-r--r-- 1 root root  868879 Jan 16 00:06 fact_sales.csv
-rw-r--r-- 1 root root  352596 Jan 15 23:21 golden_export.zip
-rw-r--r-- 1 root root  695539 Jan 15 23:15 gold_fact_sales_aggregated.csv
-rw-r--r-- 1 root root  434847 Jan 15 23:15 gold_fact_sales_aggregated.xlsx
-rw-r--r-- 1 root root    7160 Jan 15 23:15 gold_monthly_revenue_yoy.parquet
-rw-r--r-- 1 root root    5369 Jan 15 23:24 gx_data_docs.zip
drwxr-xr-x 2 root root    4096 Jan 15 20:49 

In [49]:
# Cell 4: Ingest bronze


db_path = '/content/sales_analytics/sales.duckdb'
con = duckdb.connect(db_path)

con.execute("CREATE SCHEMA IF NOT EXISTS bronze")

con.execute("CREATE OR REPLACE TABLE bronze.fact_sales AS SELECT * FROM read_csv_auto('/content/fact_sales.csv')")
con.execute("CREATE OR REPLACE TABLE bronze.dim_customer AS SELECT * FROM read_csv_auto('/content/dim_customer.csv')")
con.execute("CREATE OR REPLACE TABLE bronze.dim_product AS SELECT * FROM read_csv_auto('/content/dim_product.csv')")
con.execute("CREATE OR REPLACE TABLE bronze.dim_time AS SELECT * FROM read_csv_auto('/content/dim_time.csv')")
con.execute("CREATE OR REPLACE TABLE bronze.dim_location AS SELECT * FROM read_csv_auto('/content/dim_location.csv')")

con.close()
print("Bronze cargado.")

Bronze cargado.


In [50]:
# Cell 5: sources.yml
import os
os.makedirs('models/sources', exist_ok=True)
with open('models/sources/sources.yml', 'w') as f:
    f.write("""
version: 2

sources:
  - name: bronze
    schema: bronze
    tables:
      - name: fact_sales
      - name: dim_customer
      - name: dim_product
      - name: dim_time
      - name: dim_location
""")
print("sources.yml creado.")

sources.yml creado.


In [51]:
# CelL 6: Silver models
import os

silver_dir = 'models/silver'
os.makedirs(silver_dir, exist_ok=True)

# silver_fact_sales.sql
with open('models/silver/silver_fact_sales.sql', 'w') as f:
    f.write("""
{{ config(materialized='incremental', unique_key='sale_id', incremental_strategy='delete+insert') }}

SELECT
    sale_id,
    time_id,
    customer_id,
    product_id,
    location_id,
    quantity,
    price,
    discount,
    tax_rate,
    sale_timestamp,
    revenue_before_discount,
    revenue,
    -- Cleaning
    CASE WHEN quantity < 0 THEN 0 ELSE quantity END AS cleaned_quantity,
    CASE WHEN location_id < 1 THEN 1 ELSE location_id END AS cleaned_location_id,  -- Fix invalid
    CASE WHEN revenue <= 0 THEN revenue_before_discount ELSE revenue END AS cleaned_revenue
FROM {{ source('bronze', 'fact_sales') }}
{% if is_incremental() %}
WHERE time_id > (SELECT MAX(time_id) FROM {{ this }})
{% endif %}
""")

print("silver_fact_sales.sql actualizado con incremental_strategy='delete+insert'.")

# Silver dims
dims = ['customer', 'product', 'time', 'location']
for dim in dims:
    with open(os.path.join(silver_dir, f'silver_dim_{dim}.sql'), 'w') as f:
        f.write("""
{{ config(materialized='table') }}

SELECT *
FROM {{ source('bronze', 'dim_%s') }}
""" % dim)

print("silver models create.")

silver_fact_sales.sql actualizado con incremental_strategy='delete+insert'.
silver models create.


In [52]:
# Cell 7: Gold models
import os

gold_dir = 'models/gold'
os.makedirs(gold_dir, exist_ok=True)

with open(os.path.join(gold_dir, 'gold_fact_sales_aggregated.sql'), 'w') as f:
    f.write("""
{{ config(materialized='table') }}

SELECT
    t.date AS date_key,
    l.region AS region,
    c.segment AS customer_segment,
    p.category AS product_category,
    SUM(f.cleaned_quantity) AS total_quantity,
    SUM(f.cleaned_revenue) AS total_revenue,
    AVG(f.discount) AS avg_discount,
    SUM(f.cleaned_revenue - (p.cost * f.cleaned_quantity)) AS total_profit
FROM {{ ref('silver_fact_sales') }} f
JOIN {{ ref('silver_dim_time') }} t ON f.time_id = t.time_id
JOIN {{ ref('silver_dim_location') }} l ON f.cleaned_location_id = l.location_id
JOIN {{ ref('silver_dim_customer') }} c ON f.customer_id = c.customer_id
JOIN {{ ref('silver_dim_product') }} p ON f.product_id = p.product_id
GROUP BY 1, 2, 3, 4
""")

with open(os.path.join(gold_dir, 'gold_monthly_revenue_yoy.sql'), 'w') as f:
    f.write("""
{{ config(materialized='view') }}

WITH monthly AS (
    SELECT
        t.year || '-' || LPAD(t.month::TEXT, 2, '0') AS month_key,
        l.region,
        SUM(f.cleaned_revenue) AS revenue
    FROM {{ ref('silver_fact_sales') }} f
JOIN {{ ref('silver_dim_time') }} t ON f.time_id = t.time_id
JOIN {{ ref('silver_dim_location') }} l ON f.cleaned_location_id = l.location_id
GROUP BY 1, 2
)
SELECT
    month_key,
    region,
    revenue,
    LAG(revenue) OVER (PARTITION BY region ORDER BY month_key) AS prev_revenue,
    (revenue - LAG(revenue) OVER (PARTITION BY region ORDER BY month_key)) / NULLIF(LAG(revenue) OVER (PARTITION BY region ORDER BY month_key), 0) * 100 AS yoy_growth
FROM monthly
""")

print("Gold models Create.")

Gold models Create.


In [53]:
# Cell 8: Tests dbt
with open('models/schema.yml', 'w') as f:
    f.write("""
version: 2

models:
  - name: silver_fact_sales
    columns:
      - name: sale_id
        tests:
          - unique
          - not_null
      - name: cleaned_quantity
        tests:
          - dbt_utils.expression_is_true:
              expression: ">= 0"
      - name: discount
        tests:
          - dbt_utils.expression_is_true:
              expression: "BETWEEN 0 AND 1"
      - name: cleaned_revenue
        tests:
          - dbt_utils.expression_is_true:
              expression: "> 0"
""")

print("schema.yml actualizado con sintaxis correcta.")

schema.yml actualizado con sintaxis correcta.


In [54]:
# Cell 9:  Packages (dbt_utils)
with open('packages.yml', 'w') as f:
    f.write("""
packages:
  - package: dbt-labs/dbt_utils
    version: 1.1.1
""")

!dbt deps
print("dbt_utils instalado.")

00:06:52  Running with dbt=1.11.2
00:06:53  Installing dbt-labs/dbt_utils
00:06:53  Installed from version 1.1.1
00:06:53  Updated version available: 1.3.3
00:06:53  
00:06:53  Updates available for packages: ['dbt-labs/dbt_utils']                 
Update your versions in packages.yml, then run dbt deps
dbt_utils instalado.


In [55]:
# Cell 10: Run dbt (with cleanup)
# 1. Clean up previous builds

!rm -rf target dbt_packages

# 2. Verify Configuration
!dbt debug

# 4. Execute Models
!dbt run

# 3. Execute Tests
!dbt test

# 4. Generate Documentation
!dbt docs generate


# 5. Compress the target folder to dowload it
!zip -r dbt_docs.zip target

# 6. Dowload the Zip ;)
from google.colab import files
files.download('dbt_docs.zip')

00:07:00  Running with dbt=1.11.2
00:07:00  dbt version: 1.11.2
00:07:00  python version: 3.12.12
00:07:00  python path: /usr/bin/python3
00:07:00  os info: Linux-6.6.105+-x86_64-with-glibc2.35
00:07:00  Using profiles dir at /content/sales_analytics
00:07:00  Using profiles.yml file at /content/sales_analytics/profiles.yml
00:07:00  Using dbt_project.yml file at /content/sales_analytics/dbt_project.yml
00:07:00  adapter type: duckdb
00:07:00  adapter version: 1.10.0
00:07:00  Configuration:
00:07:00    profiles.yml file [OK found and valid]
00:07:00    dbt_project.yml file [OK found and valid]
00:07:00  Required dependencies:
00:07:00   - git [OK found]

00:07:00  Connection:
00:07:00    database: sales
00:07:00    schema: main
00:07:00    path: /content/sales_analytics/sales.duckdb
00:07:00    config_options: None
00:07:00    extensions: None
00:07:00    settings: {}
00:07:00    external_root: .
00:07:00    use_credential_provider: None
00:07:00    attach: None
00:07:00    filesystem

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Great Expectations – Validación de Calidad de Datos** 🐱‍👤💽📋


> Here we implement **Great Expectations** on top of the dbt-transformed gold layers to enforce data quality rules.  

> The validation intentionally fails on synthetic anomalies (negative values, invalid keys, zero metrics), showcasing proactive governance, error detection, and alignment with Medallion Architecture best practices.



In [56]:
!pip show great-expectations

Name: great_expectations
Version: 1.11.0
Summary: Always know what to expect from your data.
Home-page: https://greatexpectations.io
Author: The Great Expectations Team
Author-email: team@greatexpectations.io
License: Apache-2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: altair, cryptography, jinja2, jsonschema, marshmallow, mistune, numpy, packaging, pandas, pydantic, pyparsing, python-dateutil, requests, ruamel.yaml, scipy, tqdm, typing-extensions, tzlocal
Required-by: 


In [57]:
# Cell 11: Great Expectations
import great_expectations as gx
import pandas as pd
import json

# 1.Prepare the data
df_fact_sales = pd.read_csv('/content/fact_sales.csv')
if 'sale_timestamp' in df_fact_sales.columns:
    df_fact_sales['sale_timestamp'] = pd.to_datetime(df_fact_sales['sale_timestamp'])

# 2. Inicialize the context
context = gx.get_context()

# 3. Cofigure Datasource And the Asset
datasource_name = "erp_datasource"
try:
    context.data_sources.delete(datasource_name)
except:
    pass

datasource = context.data_sources.add_pandas(name=datasource_name)
data_asset = datasource.add_dataframe_asset(name="fact_sales_asset")

# --- KEy STEP👀 GX 1.11: Crear el Batch Definition ---
batch_def_name = "fact_sales_batch_def"
#The Asset now generates the lot definition
batch_definition = data_asset.add_batch_definition_whole_dataframe(name=batch_def_name)

# 4. Configure the Expectation Suite
suite_name = "erp_quality_suite_advanced"
try:
    context.suites.delete(suite_name)
except:
    pass

suite = context.suites.add(gx.ExpectationSuite(name=suite_name))

# 5. Add the Expectations
suite.add_expectation(gx.expectations.ExpectColumnValuesToNotBeNull(column="sale_id"))
suite.add_expectation(gx.expectations.ExpectColumnValuesToBeUnique(column="sale_id"))
suite.add_expectation(gx.expectations.ExpectColumnValuesToBeBetween(column="quantity", min_value=-10, max_value=10000))
suite.add_expectation(gx.expectations.ExpectColumnValuesToBeBetween(column="price", min_value=0, max_value=10000000))
suite.add_expectation(gx.expectations.ExpectColumnValuesToBeBetween(column="discount", min_value=0, max_value=0.5))
suite.add_expectation(gx.expectations.ExpectColumnValuesToBeBetween(column="tax_rate", min_value=0.01, max_value=0.2))
suite.add_expectation(gx.expectations.ExpectColumnMeanToBeBetween(column="revenue", min_value=0, max_value=10000))
suite.add_expectation(gx.expectations.ExpectCompoundColumnsToBeUnique(column_list=["sale_id", "time_id"]))
suite.add_expectation(gx.expectations.ExpectTableRowCountToBeBetween(min_value=5000, max_value=110000))

# 6.VALIDATION DEFINITION (Now using BatchDefinition)
validation_def_name = "erp_validation"
try:
    context.validation_definitions.delete(validation_def_name)
except:
    pass

validation_definition = context.validation_definitions.add(
    gx.ValidationDefinition(
        name=validation_def_name,
        data=batch_definition,  # <-- BatchDefinition
        suite=suite
    )
)

# 7. Creaate Checkpoint y execute
checkpoint_name = "erp_checkpoint"
try:
    context.checkpoints.delete(checkpoint_name)
except:
    pass

checkpoint = context.checkpoints.add(
    gx.Checkpoint(
        name=checkpoint_name,
        validation_definitions=[validation_definition]
    )
)

#
results = checkpoint.run(batch_parameters={"dataframe": df_fact_sales})

# 8. Show the results
print(f"\n¿Validation succesfull?: {results.success}")

# 9. Data Docs
context.build_data_docs()
docs_path = context.get_docs_sites_urls()[0]["site_url"]
print(f"Generate Repor: {docs_path}")

INFO:great_expectations.data_context.types.base:Created temporary directory '/tmp/tmp6czkmnp6' for ephemeral docs site
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Calculating Metrics:   0%|          | 0/49 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)



¿Validación exitosa?: True
Reporte generado en: file:///tmp/tmp6czkmnp6/index.html


In [58]:
# Cell 11: Compress and dowload Data Docs
!zip -r gx_data_docs.zip /tmp/tmp3ew5cw23/index.html   # Ajusta el nombre si es diferente (mira el INFO)
from google.colab import files
files.download('gx_data_docs.zip')

updating: tmp/tmp3ew5cw23/index.html (deflated 72%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [59]:

#Cell 12:
# Show details of each expectation (failed and successful)
print("\n=== Detalles de Expectations ===")

# 'results' is a CheckpointResult object
# Iterate through each ValidationResult within the CheckpointResult
for validation_id, validation_result_obj in results.run_results.items():
    print(f"\n--- Ejecución de Validación: {validation_id.run_id.run_name} ---")
    print(f"  Éxito General: {validation_result_obj.success}")


   # iterate trhough each ExpectationValidationResult within the ValidationResult
    for evr in validation_result_obj.results:
        # Correctly accessing the type of expectation and individual success
        exp_type = evr.expectation_config.type
        success = evr.success

        if not success:
            print(f"\n[FALLÓ] {exp_type}")
            print(f"   - Columna: {evr.expectation_config.kwargs.get('column', 'N/A')}")
            print(f"   - Conteo inesperado: {evr.result.get('unexpected_count', 'N/A')}")
            print(f"   - Valor observado: {evr.result.get('observed_value', 'N/A')}")
            print(f"   - Detalles: {evr.result.get('details', 'N/A')}")
        else:
            print(f"[PASÓ] {exp_type}")


=== Detalles de Expectations ===

--- Ejecución de Validación: None ---
  Éxito General: True
[PASÓ] expect_column_values_to_not_be_null
[PASÓ] expect_column_values_to_be_unique
[PASÓ] expect_column_values_to_be_between
[PASÓ] expect_column_values_to_be_between
[PASÓ] expect_column_values_to_be_between
[PASÓ] expect_column_values_to_be_between
[PASÓ] expect_column_mean_to_be_between
[PASÓ] expect_compound_columns_to_be_unique
[PASÓ] expect_table_row_count_to_be_between


# **7. Download Gold Layers**

In [60]:

# Create Folder for exporting Gold Layers
export_dir = '/content/golden_export'
os.makedirs(export_dir, exist_ok=True)
print(f"Folder for exporting gold layers: {export_dir}")

Folder for exporting gold layers: /content/golden_export


In [61]:


# Conect DuckDB (your actual path)
con = duckdb.connect('/content/sales_analytics/sales.duckdb')

# Gold model list
gold_models = [
    'gold_fact_sales_aggregated',
    'gold_monthly_revenue_yoy'

]

for model_name in gold_models:
    # Export to Parquet
    parquet_path = f'{export_dir}/{model_name}.parquet'
    con.execute(f"COPY {model_name} TO '{parquet_path}' (FORMAT PARQUET)")

    # Exporta to CSV
    csv_path = f'{export_dir}/{model_name}.csv'
    con.execute(f"COPY {model_name} TO '{csv_path}' (FORMAT CSV, HEADER, DELIMITER ',')")

    print(f"Export: {model_name}")
    print(f"  → Parquet: {parquet_path}")
    print(f"  → CSV: {csv_path}")

print("\n¡Export Complete!")

Export: gold_fact_sales_aggregated
  → Parquet: /content/golden_export/gold_fact_sales_aggregated.parquet
  → CSV: /content/golden_export/gold_fact_sales_aggregated.csv
Export: gold_monthly_revenue_yoy
  → Parquet: /content/golden_export/gold_monthly_revenue_yoy.parquet
  → CSV: /content/golden_export/gold_monthly_revenue_yoy.csv

¡Export Complete!


In [62]:
# List files generated
!ls -lh /content/golden_export

# Compress and Download folder
!zip -r golden_export.zip /content/golden_export
from google.colab import files
files.download('golden_export.zip')

total 884K
-rw-r--r-- 1 root root 680K Jan 16 00:07 gold_fact_sales_aggregated.csv
-rw-r--r-- 1 root root 181K Jan 16 00:07 gold_fact_sales_aggregated.parquet
-rw-r--r-- 1 root root  12K Jan 16 00:07 gold_monthly_revenue_yoy.csv
-rw-r--r-- 1 root root 7.0K Jan 16 00:07 gold_monthly_revenue_yoy.parquet
updating: content/golden_export/ (stored 0%)
updating: content/golden_export/gold_monthly_revenue_yoy.parquet (deflated 35%)
updating: content/golden_export/gold_monthly_revenue_yoy.csv (deflated 63%)
updating: content/golden_export/gold_fact_sales_aggregated.csv (deflated 74%)
updating: content/golden_export/gold_fact_sales_aggregated.parquet (deflated 14%)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## **Key Takeaways for Modern Data Platforms**
- Scalable ELT with dbt + Medallion/Kimball
- Built-in governance via dbt tests + Great Expectations
- Export-ready gold layers for BI/ML
- CI/CD ready (GitHub Actions workflow included)

# **HAVE FUN CREATING🤗**

*If this project has helpful to you, stop by my Linkedin ad let me know how it helped! ☕*

* **linkedIn:** https://www.linkedin.com/in/cristhianandrescalleseverino/

I've left the following cells open for you to add your personalized queries. Go ahead and **Create**.

***Being 1% better every day is the key 🔥***